# Elementwise Max Fusion model testing (oversampler, cosine scheduling, BCE loss):

In [1]:
import sys
root = '../../'
sys.path.append(root)   # Done to be able to import the packages and functions

import Utils.hico_evaluation.evaluation as ev
from Utils.custom_sampler import OverSampler
from Utils.custom_loss import MaskedBCELoss
from Utils.annotation_preprocessing import _load_csv_to_tensor
from Utils.train_val_split import train_val_split_hico
from hoi_classifiers import ElementwiseMaxFusionModel

import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

import random

In [2]:
seed = 42   #note that the model parameters will still be randomly initiated
torch.manual_seed(seed)
random.seed(seed)

### Loading the training set:


In [3]:
# Loading the data:
X_train = torch.load(root + "Embeddings/Combined_Embeddings/train.pt")
y_train = _load_csv_to_tensor(root + "anno/added/anno_augmented_train.csv").T # Transpose to make both first dimensions the #samples.
y_train[y_train.isnan()] = -1

In [4]:
X_test = torch.load(root + "Embeddings/Combined_Embeddings/test.pt")
y_test = _load_csv_to_tensor(root + "anno/added/anno_augmented_test.csv").T # Transpose to make both first dimensions the #samples.
y_test[y_test.isnan()] = -1

#### Training Preparations:

In [5]:
train_dataset = TensorDataset(X_train, y_train)

In [6]:
# Batch size:
bs = 512

sampler = OverSampler(y_train[:,:600], shuffle=True)

train_dataloader = DataLoader(
    train_dataset, batch_size=bs, num_workers=4, sampler=sampler)

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
model = ElementwiseMaxFusionModel(512,655,797)
model = model.to(device)

In [9]:
classweights = torch.cat((torch.ones(600),torch.ones(197)*0.5)).to(device)      # The hoi classes weigh twice as much as the seperate classes

criterion = MaskedBCELoss(ignore_label=0, convert_target_to_01= True, weight=classweights)

optimizer = torch.optim.Adam(params=model.parameters(), lr=0.0005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=5, T_mult=1)

c:\Users\rfigg\.conda\envs\PyTorch\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


#### Training loop:

In [10]:
num_epochs = 67
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Compute the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

        running_loss += loss.item()
    
    scheduler.step()
    

    print(f"Epoch {epoch+1:0{len(str(num_epochs))}}/{num_epochs}, Loss: {running_loss/len(train_dataloader):.12f}")

Epoch 01/67, Loss: 0.077848937178
Epoch 02/67, Loss: 0.020752682463
Epoch 03/67, Loss: 0.018108519449
Epoch 04/67, Loss: 0.016547317678
Epoch 05/67, Loss: 0.015932375463
Epoch 06/67, Loss: 0.014741173096
Epoch 07/67, Loss: 0.012969885595
Epoch 08/67, Loss: 0.011782603625
Epoch 09/67, Loss: 0.011124689971
Epoch 10/67, Loss: 0.010848800038
Epoch 11/67, Loss: 0.010362060341
Epoch 12/67, Loss: 0.009506600612
Epoch 13/67, Loss: 0.008915891707
Epoch 14/67, Loss: 0.008582634688
Epoch 15/67, Loss: 0.008424801250
Epoch 16/67, Loss: 0.008247614767
Epoch 17/67, Loss: 0.007849684563
Epoch 18/67, Loss: 0.007541293631
Epoch 19/67, Loss: 0.007355472426
Epoch 20/67, Loss: 0.007271082622
Epoch 21/67, Loss: 0.007198768192
Epoch 22/67, Loss: 0.006958461157
Epoch 23/67, Loss: 0.006778385777
Epoch 24/67, Loss: 0.006645023168
Epoch 25/67, Loss: 0.006576225760
Epoch 26/67, Loss: 0.006555417773
Epoch 27/67, Loss: 0.006393701140
Epoch 28/67, Loss: 0.006246784600
Epoch 29/67, Loss: 0.006146259944
Epoch 30/67, L

In [11]:
X_train.shape

torch.Size([38116, 5, 512])

#### Evaluating model:

In [12]:
test_pred = model.forward(X_test.to(device))
test_map = ev.eval_vo(test_pred[:,:600].T.cpu().detach().numpy(), y_test[:,:600].T.numpy(),600)[0].mean()

In [13]:
print(test_map * 100)

54.28274365298854


In [14]:
one_shot_list = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 1))[0]
few_shot_list_5 = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 5))[0]
few_shot_list_10 = torch.where((0 < (y_train[:,:600] == 1).sum(dim=0)) & ((y_train[:,:600] == 1).sum(dim=0) <= 10))[0]

one_shot_map = ev.eval_vo(test_pred[:,one_shot_list].T.cpu().detach().numpy(), y_test[:,one_shot_list].T.numpy(),len(one_shot_list))[0].mean()
few_5_shot_map = ev.eval_vo(test_pred[:,few_shot_list_5].T.cpu().detach().numpy(), y_test[:,few_shot_list_5].T.numpy(),len(few_shot_list_5))[0].mean()
few_10_shot_map = ev.eval_vo(test_pred[:,few_shot_list_10].T.cpu().detach().numpy(), y_test[:,few_shot_list_10].T.numpy(),len(few_shot_list_10))[0].mean()

print("Few@1 mAP: ", one_shot_map * 100)
print("Few@5 mAP: ", few_5_shot_map * 100)
print("Few@10 mAp: ", few_10_shot_map * 100)

Few@1 mAP:  33.3092496197931
Few@5 mAP:  38.77657559825985
Few@10 mAp:  40.32760448146473
